# Steam Project 2025

The ultimate goal of this project is to understand what factors affect the popularity or sales of a video game. But your boss asked you to take advantage of this opportunity to analyze the video game market globally.

In [0]:
# Import usual libraries
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import split
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, dayofyear, weekofyear
from pyspark.sql.functions import col,isnan, when, count

import json

# First analysis

In [0]:
sc = spark.sparkContext

In [0]:
# Load data
df = spark.read.format("json").load("s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json")

In [0]:
type(df)

Out[27]: pyspark.sql.dataframe.DataFrame

In [0]:
# Schema
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
# Display the five first rows
df.limit(5).toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:122: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field data. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


,data,id
0,"(10, [Multi-player, Valve Anti-Cheat enabled, ...",10
1,"(1000000, [Single-player, Partial Controller S...",1000000
2,"(1000010, [Single-player, Partial Controller S...",1000010
3,"(1000030, [Multi-player, Single-player, Co-op,...",1000030
4,"(1000040, [Single-player], 0, DoubleC Games, 0...",1000040


# Clean the data

In [0]:
# Check if df without duplicates has the same number of rows as the original
df.count() == df.dropDuplicates().count()

Out[30]: True

In [0]:
# Check the missing values
def count_missing(col_name):
  return F.sum(F.col(col_name).isNull().cast("int")).alias(col_name)
  
# Then we can apply it to all columns using a list comprehension
missing_values = df.select(*[count_missing(c) for c in df.columns]).toPandas()
display(missing_values)

data,id
0,0


In [0]:
# Describe
display(df.describe())

summary,id
count,55691
mean,1025603.0926720655
stddev,522784.96832834504
min,10
max,999990


# Analysis

- ### Which publisher has released the most games on Steam?


In [0]:
# Extract "publisher" of the columns "data"
publisher_df = df.withColumn("publisher", F.col("data.publisher"))

# Groupby publisher then count the number of games order by descending
display(publisher_df.groupBy("publisher").count().orderBy("count", ascending=False).take(5))

publisher,count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141


Databricks visualization. Run in Databricks to view.

Big Fish Game is the publisher number one

- ### What are the best rated games?

In [0]:
# Extract "name","negative","positive","genre" of the columns "data"
rated_df = (
    df.withColumn("name", F.col("data.name"))
    .withColumn("negative", F.col("data.negative"))
    .withColumn("positive", F.col("data.positive"))
    .drop("data",F.col("data"))
)

type(rated_df)

# Difference between positive and negative to see the best rated
top_rated_df = rated_df.withColumn("rated",(F.col("positive") - F.col("negative"))).sort("rated",ascending=False)
display(top_rated_df.take(5))

id,name,negative,positive,rated
730,Counter-Strike: Global Offensive,787093,5943345,5156252
570,Dota 2,317916,1534895,1216979
271590,Grand Theft Auto V,213379,1229265,1015886
105600,Terraria,22380,1014711,992331
4000,Garry's Mod,29998,861240,831242


Databricks visualization. Run in Databricks to view.

"Counter Strike" is the best rate game

- ### Are there any genres that have a better positive/negative review ratio?

In [0]:
# Ratio of "positive" and "negative"
df_ratio_rated = rated_df.withColumn("rating_ratio",F.when(F.col("negative") > 0, F.col("positive") / F.col("negative")).otherwise(None)).sort("rating_ratio",ascending=False)

#Explode the column "genre"
exploded_genre_df = df.withColumn("genre", F.explode(F.split(F.col("data.genre"), ",\s*"))).drop("data",F.col("data")).withColumn("genre", F.trim(F.col("genre")))

In [0]:
# Join the ratio and the "genre"
ratio_genre_df = df_ratio_rated.join(exploded_genre_df, on="id", how="left").groupBy("genre").sum("rating_ratio")
display(ratio_genre_df.sort("sum(rating_ratio)",ascending=False).take(10))

genre,sum(rating_ratio)
Indie,197628.52578864017
Adventure,115147.83941327485
Casual,107982.78879894926
Action,107068.07071429325
Simulation,47666.70647550773
RPG,47578.15569497053
Strategy,45011.0686729944
Early Access,23576.883645264945
Free to Play,17550.377693895767
Sports,10120.119311420694


Databricks visualization. Run in Databricks to view.

The best ratio is for the independant games

- ### Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

In [0]:
# Extract "release_date" of the columns "data"
date_df = df.withColumn("release_date", F.col("data.release_date")).drop("data")

# Transform the date with the totimestamp
date_totimestamp_df = date_df.withColumn(
    "clean_date", F.to_timestamp(F.col("release_date"), format="y/M/d")
)

#Take the year then count the games for each year, display and sort by descending
year_df = date_totimestamp_df.withColumn("year", year(F.col("clean_date")))
display(year_df.groupBy("year").count().sort("year", ascending=False))

year,count
2022,7451
2021,8805
2020,8287
2019,6949
2018,7663
2017,6006
2016,4176
2015,2566
2014,1550
2013,469


Databricks visualization. Run in Databricks to view.

There are more released game in 2021 during the Covid, we can see it on the graph.

- ### How are the prizes distributed? Are there many games with a discount?

In [0]:
# Distribution of the price
# Extract the column "price"
price_df = (
  df.withColumn("price",F.col("data.price").cast(FloatType()))
  .drop(F.col("data"))
)

#Display the describe
display(price_df.describe())


summary,id,price
count,55691,55691
mean,1025603.0926720655,773.2849832109317
stddev,522784.96832834504,1093.13458272345
min,10,0.0
max,999990,99900.0


There are outliers, the maximum is really big, 99900$ and the mean is at 773$ its a lot for games let's have a look on the discount.

In [0]:
# Aanalyse the discount
# Extract the column "discount"
discount_df = (
  df.withColumn("discount",F.col("data.discount").cast(FloatType()))
  .drop(F.col("data"))
)

# Count the number of discount
count_discount_df = discount_df.filter(F.col("discount") > 0).count()
print("Number of discount :",count_discount_df)

# Count the number of games in total
count_of_games = discount_df.select(F.col("discount")).count()
print("Number of games in total:",count_of_games)

# Make the proportion in %
proportion_of_discount = (count_discount_df/count_of_games*100)
print(proportion_of_discount,"%")

Number of discount : 2518
Number of games in total: 55691
4.521376883158858 %


There are 4.5% of the games which have a discount

- ### What are the most represented languages?

In [0]:
# Extract "languages" from the columns "data"
languages_df = df.withColumn("languages", F.col("data.languages"))

# Divide languages in lists
exploded_languages_df = languages_df.withColumn("language", F.explode(F.split(F.col("languages"), ",\s*")))
exploded_languages_df = exploded_languages_df.withColumn("language", F.trim(F.col("language")))

# Count the number of games for each languages
languages_counts_df = exploded_languages_df.groupBy("language").agg(F.count("id").alias("total_games"))

# Display and sort by descending
display(languages_counts_df.orderBy(F.desc("total_games")).take(10))

language,total_games
English,55116
German,14019
French,13426
Russian,12922
Simplified Chinese,12782
Spanish - Spain,12233
Japanese,10368
Italian,9304
Portuguese - Brazil,6750
Korean,6600


Databricks visualization. Run in Databricks to view.

The most represented languages is English then German and French.

- ### Are there many games prohibited for children under 16/18?

In [0]:
# For loop for selected ages append to empty dict the quantity of games having that age restriction
ages = [16, 18]
required_age_dict = {}

for age in ages:
    required_age_dict[age] = df.filter(df["data.required_age" ] > age).count()

display(required_age_dict)

{16: 267, 18: 6}

98.82% of games do not have age restriction, they are for everyone as it is 0.
267 games require an age higher than 16, while only 6 require an age higher than 18.

- ### What are the most represented genres?

In [0]:
# Extract "genre" of the columns "data"
genre_df = df.withColumn("genre", F.col("data.genre"))

# Transform the string to an array to explode the "genre"
genre_split_df = genre_df.withColumn("genre", F.split(F.col("genre"), ", "))
explode_genre_df = genre_split_df.withColumn("genre", F.explode(F.col("genre")))

# Group the genre then display the number of games for each genre sort by descending 
sorted_explode_genre_df = explode_genre_df.groupBy("genre").count().sort("count", ascending=False).take(5)
display(sorted_explode_genre_df)

genre,count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895


Databricks visualization. Run in Databricks to view.

The top 5 of represented genre or : Indie / Action / Casual / Adventure / Strategy

- ### Do some publishers have favorite genres?

In [0]:
# Extract the column "publisher"
publisher_favorite = df.withColumn("publisher", F.col("data.publisher")).drop(F.col("data"))
publisher_favorite_df = exploded_genre_df.join(publisher_favorite, on="id", how="inner")

In [0]:
publisher_favorite_df.groupBy("genre", "publisher").count().sort(F.desc("count")).show(50)

+---------+--------------------+-----+
|    genre|           publisher|count|
+---------+--------------------+-----+
|   Casual|      Big Fish Games|  418|
|Adventure|      Big Fish Games|  392|
|   Casual|              8floor|  202|
|      RPG|     Choice of Games|  139|
|    Indie|     Choice of Games|  136|
|   Casual|            HH-Games|  132|
|    Indie|        Laush Studio|  124|
|Adventure|     Choice of Games|  112|
|    Indie|                    |  106|
|   Casual|Alawar Entertainment|  105|
|   Casual|       Sekai Project|   99|
|    Indie|   Sokpop Collective|   97|
| Strategy|     Slitherine Ltd.|   96|
|Adventure|Alawar Entertainment|   95|
|    Indie|      Reforged Group|   88|
|    Indie|       Sekai Project|   88|
|    Indie|    Devolver Digital|   87|
|   Casual|        Laush Studio|   87|
|    Indie|             PLAYISM|   81|
|   Action|                SEGA|   80|
|    Indie|     Plug In Digital|   79|
|    Indie|        Hosted Games|   79|
|    Indie|      Piece Of

Yes, for example BigFishGame preferes Casual and Adventure and Choice of Games preferes RPG and Independants games

- ### What are the most lucrative genres?

In [0]:
# Extract the "initialprice" and "price" columns
lucratives_genre_df = (
    df.withColumn("price", F.col("data.price").cast(FloatType()))
    .drop("data")
)

# Join the exploded genre and the lucrative dataframe
lucratives_genre_df = exploded_genre_df.join(lucratives_genre_df, on="id", how="inner")
lucratives_genre_df = lucratives_genre_df.groupBy("genre").sum("price").sort("sum(price)",ascending=False)
display(lucratives_genre_df.take(10))

genre,sum(price)
Indie,2.6063036E7
Action,1.8358769E7
Adventure,1.7158179E7
Casual,1.2383583E7
Simulation,9851654.0
Strategy,9157201.0
RPG,8621295.0
Early Access,5375795.0
Sports,2385529.0
Racing,1771645.0


Databricks visualization. Run in Databricks to view.

The most lucrative is the independants games

- ### Are most games available on Windows/Mac/Linux instead ?

In [0]:
# We can see that the platforms are dictionnarys so we need to unpacked the columns from "platforms"
df_unpacked = df.select(
    "data",
    F.col("id").alias("id"),
    F.col("data.platforms.linux").alias("linux").cast(IntegerType()),
    F.col("data.platforms.mac").alias("mac").cast(IntegerType()),
    F.col("data.platforms.windows").alias("windows").cast(IntegerType()),
)

# Drop the column data
df_unpacked = df_unpacked.drop("data")

# Add an aggregation function to make the sum of each platforms, pivot it
platforms_df = platforms_df.select(
    F.lit("linux").alias("platform"), F.col("sum_linux").alias("total_users")
).union(
    platforms_df.select(F.lit("mac").alias("platform"), F.col("sum_mac").alias("total_users"))
).union(
    platforms_df.select(F.lit("windows").alias("platform"), F.col("sum_windows").alias("total_users"))
)

display(platforms_df)

platform,total_users
linux,8458
mac,12770
windows,55676


Databricks visualization. Run in Databricks to view.

Games are more available on windows

- ### Do certain genres tend to be preferentially available on certain platforms?

In [0]:
display(df_unpacked)

id,linux,mac,windows
10,1,1,1
1000000,0,0,1
1000010,0,0,1
1000030,0,1,1
1000040,0,0,1
1000080,0,1,1
1000100,0,0,1
1000110,0,0,1
1000130,0,1,1
1000280,0,0,1


In [0]:
# Linux
# Filter the column which have True
linux = df_unpacked.filter(F.col("linux") == True)

# Join the linux and the exploded genre dataframes
linux_tend = linux.join(exploded_genre_df, on="id", how="left")

# Display the dataframe of the count of linux games by genre
sum_linux_tend = linux_tend.groupBy("genre").count().withColumnRenamed("count", "sum_linux")

In [0]:
# Mac
# Filter the column which have True
mac = df_unpacked.filter(F.col("mac") == True)

# Join the mac and the exploded genre dataframes
mac_tend = mac.join(exploded_genre_df, on="id", how="left")

# Display the dataframe of the count of mac games by genre
sum_mac_tend = mac_tend.groupBy("genre").count().withColumnRenamed("count", "sum_mac")

In [0]:
# Windows
# Filter the column which have True
windows = df_unpacked.filter(F.col("windows") == True)

# Join the windows and the exploded genre dataframes
windows_tend = windows.join(exploded_genre_df, on="id", how="left")

# Display the dataframe of the count of windows games by genre
sum_windows_tend = windows_tend.groupBy("genre").count().withColumnRenamed("count", "sum_windows")

In [0]:
df_joined = sum_linux_tend \
    .join(sum_mac_tend, on="genre", how="inner") \
    .join(sum_windows_tend, on="genre", how="inner")

df_sorted_desc = df_joined.orderBy(df_joined["sum_linux"].desc(),
                                   df_joined["sum_mac"].desc(),
                                   df_joined["sum_windows"].desc())

display(df_sorted_desc.take(10))

genre,sum_linux,sum_mac,sum_windows
Indie,6978,9935,39676
Action,3379,4564,23755
Casual,3305,5130,22082
Adventure,3302,5039,21427
Strategy,1826,3005,10892
Simulation,1532,2439,10832
RPG,1524,2248,9533
Early Access,632,900,6145
Free to Play,474,845,3391
Racing,304,424,2154


Databricks visualization. Run in Databricks to view.

Every genres tend to be preferentially available on windows

# Conclusion

For sell a maximum of game they can : 
- Ensuring compatibility with Windows gives access to the majority of users.
- Add the 3 best langages : English, German and French.
- The Indie genres is the most represented but the best rated games are many Action games